In [537]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score

In [538]:
with open('dataframes/df2015', 'rb') as f_2015:
    df2015 = pickle.load(f_2015)
f_2015.close()

with open('dataframes/df2016', 'rb') as f_2016:
    df2016 = pickle.load(f_2016)
f_2016.close()

with open('dataframes/df2017', 'rb') as f_2017:
    df2017 = pickle.load(f_2017)
f_2017.close()

with open('dataframes/df2018', 'rb') as f_2018:
    df2018 = pickle.load(f_2018)
f_2018.close()

with open('dataframes/df2019', 'rb') as f_2019:
    df2019 = pickle.load(f_2019)
f_2019.close()

with open('dataframes/df2020', 'rb') as f_2020:
    df2020 = pickle.load(f_2020)
f_2020.close()

with open('dataframes/df_all', 'rb') as f_all:
    df_all = pickle.load(f_all)
f_all.close()

with open('dataframes/df_feats', 'rb') as f_feats:
    df_feats = pickle.load(f_feats)
f_feats.close()

# Linear Regression
- The data in this set lends itself to linear regression since the happiness score is positively correlated with all its features as seen in the EDA steps. 
- GDP is the highest correlated features. 
- Since there are only 7 features, it's likely that we won't need to perform PCA on the dataset since there is no worthy feature to remove. These have all been engineered to be linearly correlated with our target (happiness). 
- Here, we will explore different ways regression can be applied to this dataset to make predictions.

### Question 1: What is the happines score of each country for 2020 given the 2020 features?
- we predicted for MOST countries based on SOME countries for this one

In [539]:
Y = df2020['Score'] # target 
X = df2020.iloc[:, 1:].drop(['Score'],axis='columns') # features 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=.2)
# print(train_x.shape)
# print(test_y.shape) # ok! 

In [540]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x.iloc[:, 1:])
test_x = scaler.transform(test_x.iloc[:, 1:])

In [541]:
lr_reg = LinearRegression()
lr_reg.fit(train_x, train_y) # keep the countries # index without it 
hyp = lr_reg.predict(test_x)

In [542]:
# metrics
res = test_y - hyp
r2_score = r2_score(test_y, hyp)
var = explained_variance_score(test_y, hyp)
mse = mean_squared_error(test_y, hyp)
print('R2 score...          \t', r2_score)
print('Variance explained...\t', var)
print('Mean squared error...\t', mse)
# DON'T TOUCH THIS # IT'S VERY GOOD

R2 score...          	 0.8137411138644184
Variance explained...	 0.8257758562172264
Mean squared error...	 0.1901389671095601


In [ ]:
# tell me which countries were predicted and what they are

### Question 2: What is the happiness score of each country for 2020 given the the data from past years (2015-2019)?

In [547]:
Y = df_feats['Score'] # target 
X = df_feats.iloc[:, 1:].drop(['Score'],axis='columns') # features 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=.2)
# print(train_x.shape)
# print(test_y.shape) # ok! 

In [544]:
# scaler = StandardScaler()
# train_x = scaler.fit_transform(train_x.iloc[:, 1:])
# test_x = scaler.fit(test_x.iloc[:, 1:])

In [550]:
train_x.iloc[:, 2:]

,GDP per capita,Social support,Healthy life expectancy,Freedom,Perceptions of corruption,Generosity
138,0.275000,0.572000,0.410000,0.293000,0.085000,0.177000
54,1.401678,1.128274,0.900214,0.257922,0.063283,0.206674
69,0.893730,1.111110,0.582950,0.462350,0.073960,0.252960
24,1.131000,1.331000,0.808000,0.431000,0.061000,0.197000
7,1.303000,1.557000,1.026000,0.585000,0.380000,0.330000
...,...,...,...,...,...,...
143,0.489000,1.169000,0.168000,0.359000,0.093000,0.107000
86,1.052000,1.538000,0.657000,0.394000,0.028000,0.244000
0,1.340000,1.587000,0.986000,0.596000,0.393000,0.153000
129,0.741900,0.385620,0.729260,0.405770,0.383310,0.055470


In [551]:
lr_reg = LinearRegression()
# lr_reg = GridSearchCV(lr, param_grid={}, n_jobs = 5)
lr_reg.fit(train_x.iloc[:, 2:], train_y) # keep the countries # index without it 
hyp = lr_reg.predict(test_x.iloc[:, 2:])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# r2_score = r2_score(test_y, hyp)
# var = explained_variance_score(test_y, hyp)
# mse = mean_squared_error(test_y, hyp)
# print('R2 score...\t', r2_score)
# print('Variance explained...\t', var)
# print('Mean squared error...\t', mse)

# References
- [Happiness Data Analysis 2015-2019](https://www.kaggle.com/kojisera/happiness-data-analysis-2015-2019) - See top happy 30 countries scatter plot for 2015-2019